In [1]:
# Import needful packages
import sys

import plotly.graph_objects as go
import pandas as pd
from scipy.stats import gmean


import plotly.io as pio
from IPython.display import display

# Set the default renderer
pio.renderers.default = "notebook"

# Manage sys.path more cleanly
aggreg_path = "/nfs/site/disks/dvsnoval_perf_01/aggreg"

# Remove existing entry and add to beginning
while aggreg_path in sys.path:
    sys.path.remove(aggreg_path)
sys.path.insert(0, aggreg_path)

# Clear import cache for fresh import
for module in ["utils.aggregator", "utils"]:
    sys.modules.pop(module, None)

# Import utilities
from utils.aggregator import interpolate_dataframe, split_dataframe_by_column, expand_dataframes_to_max  # noqa: E402

In [2]:
nvl = pd.read_csv(
    "../../Veera_data/25WW37p4_NVL-P_N2P_WW31_Emu_Aligned_Trend_For_Veera.fbt (1).csv",
    usecols=["workload_compatibility_name", "gt_slice_freq", "gt_workload_fps", "far_mem_transfer_freq", "gt_power"],
).dropna()
ptl = pd.read_csv(
    "../../Veera_data/25WW24p3_AMX2p0_PTL-P_3D_B0Si_FpsCdyn_PostSiConfigs.fbt (2).csv",
    usecols=["workload_compatibility_name", "gt_slice_freq", "gt_workload_fps", "far_mem_transfer_freq", "gt_power"],
).dropna()

nvl_aggregated = (
    nvl.groupby(["far_mem_transfer_freq", "gt_slice_freq"])
    .agg(
        {
            "gt_workload_fps": lambda x: gmean(x[x > 0]),  # Geometric mean (excluding zeros/negatives)
            "workload_compatibility_name": "count",  # Count of workloads
            "gt_power": "mean",  # Optional: regular mean for gt_power
        }
    )
    .reset_index()
)
ptl_aggregated = (
    ptl.groupby(["far_mem_transfer_freq", "gt_slice_freq"])
    .agg(
        {
            "gt_workload_fps": lambda x: gmean(x[x > 0]),  # Geometric mean (excluding zeros/negatives)
            "workload_compatibility_name": "count",  # Count of workloads
            "gt_power": "mean",  # Optional: regular mean for gt_power
        }
    )
    .reset_index()
)


ptl_frequencies = split_dataframe_by_column(ptl_aggregated, "far_mem_transfer_freq")
nvl_frequencies = split_dataframe_by_column(nvl_aggregated, "far_mem_transfer_freq")
nvl_frequencies.keys(), ptl_frequencies.keys()

(dict_keys([np.int64(7028), np.int64(8031), np.int64(9035), np.int64(10039)]),
 dict_keys([np.int64(7467), np.int64(8533), np.int64(9600)]))

In [5]:
for frequency, data in nvl_frequencies.items():
    nvl_frequencies[frequency] = interpolate_dataframe(data, "gt_power", 1)[["gt_power", "gt_workload_fps"]]
for frequency, data in ptl_frequencies.items():
    ptl_frequencies[frequency] = interpolate_dataframe(data, "gt_power", 1)[["gt_power", "gt_workload_fps"]]

# Expand dataframes for maximum gt_power so that non-existing points kept the highest FPS number
ptl_frequencies = expand_dataframes_to_max(ptl_frequencies, "gt_power", max_executive_value=40, step=1)
nvl_frequencies = expand_dataframes_to_max(nvl_frequencies, "gt_power", max_executive_value=40, step=1)

In [6]:
fig = go.Figure()

x_column = "gt_power"
y_column = "gt_workload_fps"

for freq, data in nvl_frequencies.items():
    fig.add_trace(
        go.Scatter(
            x=data[x_column],
            y=data[y_column],
            mode="markers+lines",
            name=f"NVL freq {freq}",
            opacity=0.7,
            line=dict(color="red"),
        )
    )

for freq, data in ptl_frequencies.items():
    fig.add_trace(
        go.Scatter(
            x=data[x_column],
            y=data[y_column],
            mode="markers+lines",
            name=f"PTL freq {freq}",
            opacity=0.7,
            line=dict(color="blue"),
        )
    )

fig.update_layout(title=f"Chart: {y_column} vs {x_column}", xaxis_title=x_column, yaxis_title=y_column, height=600)

display(fig)

For every PTL freq
take it as a base line and compare it against all of NVL
And put them on the three separate charts
3 charts with 4 lines
Extend last available FPS
Use 10, 15, 30 W and check which NVL is better for each baselines
Based on that select the best configuration.

## Garbage goes next